###This is Assignment No.1

Assignment 1: Build Your First Agent
Task:
Create a Personal Assistant Agent that can:
1. Have basic conversations
2. Remember user's name and preferences across sessions
3. Perform simple calculations
4. Tell current time/date

In [1]:
# Install required packages
%pip install -q langgraph langsmith langchain-openai python-dotenv

# Optional: for visualization
%pip install -q matplotlib graphviz

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import getpass
from typing import Annotated, Dict, List, Any
from typing_extensions import TypedDict

# Set up environment variables
def setup_environment():
    """Setup API keys for the tutorial"""
    
    # OpenAI API Key
    if not os.environ.get("OPENAI_API_KEY"):
        openai_key = getpass.getpass("Enter your OpenAI API Key: ")
        os.environ["OPENAI_API_KEY"] = openai_key
    
    # Optional: LangSmith for observability (highly recommended)
    if not os.environ.get("LANGSMITH_API_KEY"):
        print("LangSmith setup (optional but recommended for debugging):")
        langsmith_key = getpass.getpass("Enter your LangSmith API Key (or press Enter to skip): ")
        if langsmith_key:
            os.environ["LANGSMITH_API_KEY"] = langsmith_key
            os.environ["LANGCHAIN_TRACING_V2"] = "true"
            os.environ["LANGCHAIN_PROJECT"] = "LangGraph-Tutorial"
            print("✅ LangSmith tracing enabled!")
        else:
            print("⚠️ Skipping LangSmith setup")
    
    print("🚀 Environment setup complete!")

# Run setup
setup_environment()

LangSmith setup (optional but recommended for debugging):
✅ LangSmith tracing enabled!
🚀 Environment setup complete!


In [3]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# Define our State - this is the "memory" of our agent
class State(TypedDict):
    # messages will store our conversation history
    # add_messages is a special function that appends new messages instead of replacing them
    messages: Annotated[list, add_messages]

print("✅ State defined!")
print("Our state has one field: 'messages' that will store the conversation history")
print("The add_messages function ensures new messages are appended, not replaced")


✅ State defined!
Our state has one field: 'messages' that will store the conversation history
The add_messages function ensures new messages are appended, not replaced


### Step 2: Create the Language Model

Now we need an AI model to power our chatbot. We'll use OpenAI's GPT model.

In [5]:
from langchain_openai import ChatOpenAI

# Initialize the language model
llm = ChatOpenAI(
    model="gpt-3.5-turbo",  # You can also use "gpt-4" if you have access
    temperature=0.7,        # Controls creativity (0 = deterministic, 1 = very creative)
)

print("✅ Language model initialized!")
print(f"Model: {llm.model_name}")

# Let's test it quickly
test_response = llm.invoke("I see this is your first assignment, Good Luck !!")
print(f"Test response: {test_response.content}")

✅ Language model initialized!
Model: gpt-3.5-turbo
Test response: Thank you! I appreciate your encouragement. I'll do my best to complete the assignment successfully.


### Step 3: Create the Chatbot Node

A **node** is a function that:
1. Takes the current state as input
2. Does some work (like calling the LLM)
3. Returns updates to the state

Let's create our chatbot node:


In [6]:
def chatbot_node(state: State) -> Dict[str, Any]:
    """
    The main chatbot node that have basic conversations, processes messages and generates responses.
    
    Args:
        state: Current state containing conversation history and remembers user's name and preferences across sessions.
        
    Returns:
        Dict with new messages to add to state
    """
    # Get the conversation history from state
    messages = state["messages"]
    
    # Call the LLM with the conversation history
    response = llm.invoke(messages)
    
    # Return the new message to be added to state
    return {"messages": [response]}

print("✅ Chatbot node created!")
print("This node will:")
print("1. Take the conversation history and user's name from state")
print("2. Send it to the LLM")
print("3. Return the LLM's response to be added to state")

✅ Chatbot node created!
This node will:
1. Take the conversation history and user's name from state
2. Send it to the LLM
3. Return the LLM's response to be added to state


### Step 4: Build the Graph

Now we'll create the graph by:
1. Creating a StateGraph
2. Adding our chatbot node
3. Defining the flow (edges)
4. Compiling it into a runnable application
